In [1]:
import os
import cv2
import time
import random
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as compare_ssim

In [ ]:
# 이미지 전처리
def preprocess_img(img):
    
    # img = cv2.imread(ip[0])

    # 그레이스케일화
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 이미지 스무딩
    img_gray = cv2.GaussianBlur(img_gray, (3, 3), 0)

    return img, img_gray

In [ ]:
def preprocess_tem():
    
    template_h = "./jaesangtem/tem5.jpg"
    template_v = "./jaesangtem/tem2.jpg"
    template_v2 = "./jaesangtem/tem3.jpg"
    template_v3 = "./jaesangtem/tem4.jpg"
    
    # img = cv2.imread(ip[0])
    template_h = cv2.imread(template_h)  # 가로면 템플릿
    template_v = cv2.imread(template_v)  # 세로면 템플릿
    template_v2 = cv2.imread(template_v2)  # 세로면 템플릿
    template_v3 = cv2.imread(template_v3)  # 세로면 템플릿

    # 그레이스케일화
    template_h = cv2.cvtColor(template_h, cv2.COLOR_BGR2GRAY)
    template_v = cv2.cvtColor(template_v, cv2.COLOR_BGR2GRAY)
    template_v2 = cv2.cvtColor(template_v2, cv2.COLOR_BGR2GRAY)
    template_v3 = cv2.cvtColor(template_v3, cv2.COLOR_BGR2GRAY)

    return template_h, template_v, template_v2, template_v3

In [ ]:
class compareimg:
    def __init__(self):
        pass
    
    def readimg(self, filepath):
        img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
        
        return img
    
    def diffimg(self, img1, img2):
        orb = cv2.ORB_create()
        
        kp1, des1 = orb.detectAndCompute(img1, None)
        kp2, des2 = orb.detectAndCompute(img2, None)
        
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        
        matches = bf.match(des1, des2)
        
        matches = sorted(matches, key = lambda x:x.distance)
        
        bf = cv2.BFMatcher()
        matches = bf.knnMatch(des1, des2, k=2)
        
        good = []
        for m,n in matches:
            if m.distance < 0.85 * n.distance:
                good.append([m])
        
        knn_image = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good, None, flags=2)
        plt.figure(figsize=(30,25))
        plt.imshow(knn_image)
        plt.show()
        return knn_image
    
    def run(self):
        filepath1 = './first.jpg'
        filepath2 = './first.jpg'
        
        img1 = self.readimg(filepath1)
        img2 = self.readimg(filepath2)
        
        self.diffimg(img1, img2)
        return img1, img2
        
if __name__ == '__main__':
    clmg = compareimg()
    img1, img2 = clmg.run()

    
    (score, diff) = compare_ssim(img1, img2, full=True)
    diff = (diff*255).astype("uint8")

    print(f"Similarity: {score:.5f}")

In [ ]:
# 이미지 불러오기

def check_img(kind="overkill"):
    # random.seed(time.time_ns() % 10000)
    num_OK = 0
    num_NG = 0
    template_h, template_v, template_v2, template_v3 = preprocess_tem()
    if kind == "all":
        paths = os.listdir("./product_images/")
        img_paths = []
        for p in paths:
            file_path = "./product_images/" + p + "/"
            img_paths = list(map(lambda x: [file_path + x, p], os.listdir(file_path)))
            ok_path = os.mkdir(file_path + "result/ok")
            ng_path = os.mkdir(file_path + "result/ng")
        print(f'img_paths : {img_paths}')
        count = 0
        while count > len(ip):
            ip = random.choice(img_paths)
            img = cv2.imread(ip[0])
            img, img_gray = preprocess_img(img)
            cnt, img = match_template(img, img_gray, template_h, template_v, template_v2, template_v3)
            if cnt >= 1:
                cv2.imwrite(ng_path + ip[0], img)
                num_NG += 1
            else:
                cv2.imwrite(ok_path + ip[0], img)
                num_OK += 1
            count += 1

    else:
        file_path = "./product_images/" + kind + "/"
        img_paths = os.listdir(file_path)
        ok_path = os.mkdir(file_path + "result/ok")
        ng_path = os.mkdir(file_path + "result/ng")
        for i in range(len(img_paths)):
            img = cv2.imread(file_path + img_paths[i])
            img, img_gray = preprocess_img(img)
            cnt, img = match_template(img, img_gray, template_h, template_v, template_v2, template_v3)
            if cnt >= 1:
                cv2.imwrite(file_path + "result/ng/" + img_paths[i], img)
                num_NG += 1
            else:
                cv2.imwrite(file_path + "result/ok/" + img_paths[i], img)
                num_OK += 1
    return print(f'양품 수량 : {num_OK} \n불량품 수량 : {num_NG}')